## Titanic3 dataset analysis

Thomas Cason from the University of Virginia has greatly updated and improved the titanic data frame using the Encyclopedia Titanica and created a new dataset called TITANIC3. This dataset reflects the state of data available as of August 2, 1999. Some duplicate passengers have been dropped; many errors have been corrected; many missing ages have been filled in; and new variables have been created.

https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3info.txt

#https://biostat.app.vumc.org/wiki/Main/DataSets
#https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic.html


In [ ]:
# load the libraries
import pandas as pd  # data tools
import numpy as np  # maths
import seaborn as sns # visualizations
import missingno as msno # for NaN visualization
import matplotlib.pyplot as plt # for data visualization, graph plotting
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import uniform
import sweetviz as viz

#from scipy.stats import randint, uniform
#from sklearn import linear_model,preprocessing,tree,model_selection # for prediction models
# from sklearn.model_selection import cross_val_predict
# from sklearn.metrics import confusion_matrix
# from sklearn.naive_bayes import GaussianNB
# from sklearn.cluster import KMeans



# from sklearn.ensemble import VotingClassifier
# from sklearn.model_selection import GridSearchCV
# #from sklearn.metrics import jaccard_similarity_score


import warnings
#.................................


In [ ]:
# define the data set variables and their locations.

data_folder = './data/titanic3.xls'

data = pd.read_excel(data_folder)
print(data.shape)

## Exploratory Data Analysis

In [ ]:
from sklearn.model_selection import ShuffleSplit

In [ ]:

train, test = train_test_split(data, test_size=.2, random_state=15)

print("features train and test: ", X_train.shape, X_test.shape)
print('targets train and test: ', y_train.shape, y_test.shape)

report = viz.compare([X_train,"train"], [X_test, "test"],)

report.show_html("Report.html") # Not providing a filename will default to SWEETVIZ_REPORT.html

In [ ]:
# look at the header records for the datasets, what kind of data is there.
data.head()

analysis - the cabin number has some NAN which, if we use, will need to be dealth with.  

In [ ]:
data.info()

### Data Dictionary

VARIABLE DESCRIPTIONS:
| pclass                                             | Passenger Class   |  (1 = 1st; 2 = 2nd; 3 = 3rd)        |
| :---------------------------------------------------------------------------------------------------------- | --- | --- |
| survival   |  Survival     |   (0 = No; 1 = Yes)   |
| name          | name    |     |
| sex           | sex    |     |
| age                  | age    |     |
| sibsp                | Number of Siblings/Spouses Aboard    |     |
| parch                | Number of Parents/Children Aboard     |     |
| ticket               | Ticket Number     |     |
| fare                 | Passenger Fare     |     |
| cabin                | Cabin     |     |
| embarked                  | Port of Embarkation    | (C = Cherbourg; Q = Queenstown; S = Southampton)    |
| boat                | Lifeboat     |     |
| body                | Body Identification Number    |     |
| home.dest           | Home/Destination    |     |

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
If the Age is Estimated, it is in the form xx.5

Fare is in Pre-1970 British Pounds (£)
Conversion Factors:  1£ = 12s = 240d and 1s = 20d

With respect to the family relation variables (i.e. sibsp and parch) some relations were ignored.  The following are the definitions used for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiancées Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.


In [ ]:
data.describe()

The age runs from a few months to 80 years, siblings/spouse from 0 to 8, parents/children from 0 to 6 and fare from 0 to 512.

In [ ]:
# checking on if/where the null values are

data.isnull().sum()

cabin is missing 77% of their values and age is missing 20%

In [ ]:
msno.matrix(data)  # taking a look at the missing data in relation to the other variables in the dataset

In [ ]:
print('Missing data in each Titanic dataframe column:')
for c in data.columns:
    missing_data = len(data) - data[c].count()
 
    if (missing_data > 0 or missing_data =='NaN'):        
        print(f"    {c} is missing {str(round(float(missing_data / float(len(data))) * 100, 1))}% if its values ({missing_data})")


analysis of the missing data:

    1: age is missimg 20% of its values.  I need to see if thiss can be supplemented because this would be aa desireable feature in survavbility analysis.
    2: cabin is mostly missing its values but the deck or general location of the cabin might be able to be inferred by the fare which is missing only 1 value    
    3: body number are mostly missing their data.  With over 50% of their data missing and unless this can be supplemented, its data will be dropped.
    4: boat is missing 63% of its data, let's see if this can be supplemented in some manner to see if it adds anything to the prediction.  
    4: home/destination is missing 43% of its data, we'll keep it in the dataset so othe ranalysis can be performed.

In [ ]:
cols_num = data.select_dtypes(exclude="object")
cols_cat = data.select_dtypes(include="object")


In [ ]:
uniques = {col:data[col].nunique() for col in cols_cat}  # retrieve unique values for each of the categeorical columns
for key in uniques:
    print(f"{key} has {uniques[key]} unique values")

In [ ]:
# age is usually an important indicator, we should impute the missing values for age:
# copy the data: 

titanic = data.copy()
titanic['age'] = titanic['age'].fillna(titanic['age'].mean())
titanic.head()

# there are other ways that this could have been solved but due to the small dataset size, getting more precise here would likely have little return value in the end

In [ ]:
# the cabin is missing 77% of its values.  one method to solve IS to use the fare column.  more than likely the fare across cabins was similar

# create a new dataframe which only includes the available cabin numbers
cabinfare = titanic[['cabin', 'fare']]
cabinfare = cabinfare[cabinfare['cabin'].notna()]
cabinfare.reset_index(drop=True, inplace=True)
#cabinfare.head()


In [ ]:
x = ''

for i in range(0, (len(titanic.cabin))):
    cabin = titanic.cabin[i]
    
    if str(cabin) == "nan":
        val = cabinfare.iloc[(cabinfare['fare']-titanic.fare[i]).abs().argsort()[:1]]  # the :1 says send me back the closest record
        titanic.at[i,'cabin'] = str(val.iat[0,0])

regex = "(?P<deck>[A-Z])(?P<Number>[0-9]*)"
tmp = titanic['cabin'].str.extract(regex, expand=False)
titanic['deck'] = tmp['deck'].fillna('Z')

print(f"Unique FN: {titanic['deck'].unique()}")



In [ ]:
# next is to fix of 'embarked'
# 1) since most of the passengers embarked in southhampton, we should fill the emties in with 'S'%%!
# 2) drop the boat and body columns, too much missing data in each

titanic["embarked"].fillna("S", inplace = True)
titanic.drop(['boat', 'body', 'name', 'ticket', 'fare', 'cabin', 'home.dest'], axis=1, inplace=True)
titanic.head()

In [ ]:
# change the keys to make them more descriptive

def label_survive(row):
    if row['survived'] == 1:
        return 'yes'
    if row['survived'] == 0:
        return 'no'
    else:    
        return 'unk'

        
def label_familysize(row):
    if row['sibsp'] + row['parch'] == 0:
        return 0
    if row['sibsp'] + row['parch'] == 1:
        return 1
    if row['sibsp'] + row['parch'] == 2:
        return 2
    else:    
        return 3

# embarked keys:
titanic.loc[:,'embarked'].replace(['C','S','Q'], ['Cherbourg','Southampton','Queenstown'], inplace=True)

# survived keys:
#titanic.loc[:,'survived'].replace([0,1],['no','yes'], inplace=True)
titanic['survived label'] = titanic.apply(lambda row: label_survive(row), axis = 1)
titanic['companions'] = titanic.apply(lambda row: label_familysize(row), axis = 1)



In [ ]:
# looking at age in detail
# Get summary descriptive statistics
df_describe = pd.DataFrame(titanic['age'].describe())

#Change the index labels and round the values reported
df_describe.index = ['population size', 'mean', 'std. dev', 'min', '25% qt', 'median', '75% qt', 'max']
df_describe = df_describe.round(decimals=3)
df_describe

In [ ]:

# fig, axs = plt.subplots(1, 2, figsize=( 12, 6))
# sns.histplot(x= titanic['age'],  kde=True, ax = axs[0])
# sns.violinplot(x= titanic['age'],  ax = axs[1])

fig, axs = plt.subplots(1, 3, figsize=( 25, 6))
sns.histplot(x= titanic['age'],  kde=True, ax = axs[0])
sns.violinplot(x= titanic['age'],  ax = axs[1])

age_bins = np.arange(0, titanic['age'].max()+5, 5) #np.arange(0, 80, 4)
sns.histplot(titanic.loc[(titanic['survived label']=='no') & (~titanic['age'].isnull()),'age'], bins=age_bins, color='#d62728',  kde=True,   ax = axs[2])
sns.histplot(titanic.loc[(titanic['survived label']=='yes') & (~titanic['age'].isnull()),'age'], bins=age_bins,  kde=True,   ax = axs[2])
plt.title('age distribution among survival classes')
plt.ylabel('absolute frequency')
plt.legend(['did not survive', 'survived'])
plt.show()



In [ ]:

# for the age cohorts ===============
bins = [0, 5, 14, 25, 35, 60, np.inf]
labels = ['toddler', 'child', 'teenager', 'young adult', 'adult', 'senior']
titanic['age cohort'] = pd.cut(titanic["age"], bins, labels = labels)

# for the columns being used in the charts
cols = ['pclass', 'survived', 'sex',  'sibsp', 'parch', 'embarked', 'deck','age']

vz_rows = 2
vz_cols = 4

fig, axs = plt.subplots(vz_rows, vz_cols, figsize=( vz_cols * 6, vz_rows * 6))

for r in range(0, vz_rows):
    for c in range(0, vz_cols):
        if r==1 and c==3: # draw the last chart differently from the others            
            ax = axs[r][c]
            sns.barplot(x="age cohort", y="survived", data=titanic, ax = ax)
            #ax.set_title(cols[i], fontsize= 14, fontweight= 'bold')
            ax.legend(title= 'survival amongst age cohorts', loc= 'upper center')
        else:
            i = r * vz_cols + c
            ax = axs[r][c]
            sns.countplot(x= titanic[cols[i]], hue=titanic['survived'],  ax = ax)        
            ax.set_title(cols[i], fontsize= 14, fontweight= 'bold')
            ax.legend(title= 'survived', loc= 'upper center')
plt.tight_layout()



analysis: for the age groups, the youngest ages had the best chance of survival while adults experienced a better surivial rate than either teenagers or young adults.  

The charts above show that a male travelling without family and not in first class would have a poor chance of surviving the sinking.


In [ ]:
# look at survivorship amongst various groups
# age / pclass
# age / sex
# pclass / family


cols1 = ['pclass', 'sex',  'companions', 'deck', 'pclass', 'sex','deck', 'embarked' ]
cols2 = ['age', 'age',  'age', 'age', 'companions', 'companions','companions', 'age']

vz_rows = 2
vz_cols = 4

fig, axs = plt.subplots(vz_rows, vz_cols, figsize=( vz_cols * 6, vz_rows * 6))

for r in range(0, vz_rows):
    for c in range(0, vz_cols):        
        i = r * vz_cols + c
        ax = axs[r][c]
        sns.violinplot(x= titanic[cols1[i]], y=titanic[cols2[i]], hue=titanic['survived'], split=True, palette={0: 'r', 1: 'g'}, ax = ax)   
        #ax.set_title(cols[i], fontsize= 14, fontweight= 'bold')
        #ax.legend(title= 'survived', loc= 'upper center')
plt.tight_layout()



#sns.violinplot(x=’Pclass’, y=’Age’, hue=’Survived’, split=True, data=train_data, palette={0: “r”, 1: “g”});


In [ ]:

pclass = [1,2,3]
# ageLbound = [60, 40, 20]
# ageUbound = [99, 60, 40]

#ageLbound = [70,65,60,55,50,45,40,35,30,25,20,15,10,5,0]
#ageUbound = [99,70,65,60,55,50,45,40,35,30,25,20,15,10,5]

ageLbound = [70,60,50,40,30,20,10,0]
ageUbound = [99,70,60,50,40,30,20,10]


sex = ['male', 'female']

df_stat = pd.DataFrame(columns = ['class', 'sex', 'age', 'surv_pct'])

def div(x,y):
    if y == 0: return 0
    return x / y

for c in range(0, 3):
    for a in range(0, 8):
        for s in range(0,2):

            rate = round(div(len(titanic[(titanic['pclass']==pclass[c]) & (titanic['age']>ageLbound[a]) & (titanic['age']<=ageUbound[a]) \
                        & (titanic['survived']==1) & (titanic['sex']==sex[s]) ]) , len(titanic[(titanic['pclass']==pclass[c]) & \
                        (titanic['age']>ageLbound[a]) & (titanic['age']<=ageUbound[a]) & (titanic['sex']==sex[s]) ]))*100,1)

            df_stat = df_stat.append({'class':pclass[c], 'sex':sex[s], 'age':ageLbound[a], 'surv_pct':rate}, ignore_index=True)
            

#sns.lineplot(x= 'class', y= 'surv_pct', hue='sex', data=df_stat)

 

In [ ]:

sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.lineplot( x = 'age',
             y = 'surv_pct',
             hue = 'sex',
             style= 'class',
             data = df_stat)

             

analysis: although close to be confusing, the chart above shows a male in the third class, over the age of ten had a low chance to survive the sinking.  a female, above the age of ten and in first class had the best chance to survive.

In [ ]:
# remove redundant features
titanic.drop(['age', 'survived label'], axis=1, inplace=True)
titanic.head()

## create the models
1) logistic regression

In [ ]:
features = titanic.drop(['survived'], axis=1)
target = titanic['survived']


In [ ]:
features = pd.get_dummies(features)
#features.head()

In [ ]:
mask = np.zeros_like(features.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(15,15))
sns.heatmap(features.corr(),
            cmap='coolwarm', mask=mask)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=.2, random_state=15)
print("features train and test: ", X_train.shape, X_test.shape)
print('targets train and test: ', y_train.shape, y_test.shape)


In [ ]:
# report = viz.compare([X_train,"train"], [X_test, "test"],)

# report.show_html("Report.html") # Not providing a filename will default to SWEETVIZ_REPORT.html


## Cross-validation 

Evaluate, using cross-validation, which of the following classifiers performs best with our training data:

Decision Tree
Random Forest

Extra Trees

AdaBoost

Logistic Regression

K-nearest neighbors

SVC

Gradient Boosting

eXtreme Gradient Boosting

In [ ]:
random_state=15

# Scale features such that the mean is 0 and standard deviation is 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Number of cross-validation folds
k_folds = 10

# Number of estimators for tree-based ensembles
n_estimators = 100

# Create a dictionary containing the instance of the models, scores, mean accuracy and standard deviation
classifiers = {
    'name': ['DecisionTree', 'RandomForest', 'ExtraTrees', 'AdaBoost', 'LogReg', 'KNN', 'SVC',
             'XGBoost', 'GradientBoost'],
    'models': [DecisionTreeClassifier(random_state=random_state),
               RandomForestClassifier(random_state=random_state, n_estimators=n_estimators),
               ExtraTreesClassifier(random_state=random_state, n_estimators=n_estimators),
               AdaBoostClassifier(random_state=random_state, n_estimators=n_estimators),
               LogisticRegression(random_state=random_state),
               KNeighborsClassifier(),
               SVC(random_state=random_state),
               XGBClassifier(random_state=random_state, n_estimators=n_estimators),
               GradientBoostingClassifier(random_state=random_state, n_estimators=n_estimators)], 
    'scores': [],
    'acc_mean': [],
    'acc_std': []
}

# Run cross-validation and store the scores
for model in classifiers['models']:
    score = cross_val_score(model, X_train, y_train, cv=k_folds, n_jobs=4)
    classifiers['scores'].append(score)
    classifiers['acc_mean'].append(score.mean())
    classifiers['acc_std'].append(score.std())    

# Create a nice table with the results
classifiers_df = pd.DataFrame({
    'Model Name': classifiers['name'],
    'Accuracy': classifiers['acc_mean'],
    'Std': classifiers['acc_std']
}, columns=['Model Name', 'Accuracy', 'Std']).set_index('Model Name')

classifiers_df.sort_values('Accuracy', ascending=False)

## let's tune the top three

In [ ]:

from script_step2 import train_evaluate

In [ ]:
import skopt

from script_step2 import train_evaluate

SPACE = [
    skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),
    skopt.space.Integer(1, 30, name='max_depth'),
    skopt.space.Integer(2, 100, name='num_leaves'),
    skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),
    skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')]


@skopt.utils.use_named_args(SPACE)
def objective(**params):
    return -1.0 * train_evaluate(params)


results = skopt.forest_minimize(objective, SPACE, n_calls=30, n_random_starts=10)
best_auc = -1.0 * results.fun
best_params = results.x

print('best result: ', best_auc)
print('best parameters: ', best_params)


In [ ]:
# SVC



## 5.2 Tuning hyperparameters 

To further improve the models we can tune their hyperparameters using randomized parameter optimization or grid search. 

I chose randomized parameter optimization because it typically performs just as well as grid search but with much fewer iterations, furthermore, the number of iterations in a parameter that we control. 

Obviously, more iterations are always better but if we want to make quick tests we can easily reduce the tuning time by lowering this parameter instead of reducing all of the hyperparameter search ranges.

In [ ]:
# Utility function to report best scores
# Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py
def report(results, n_top=3, limit=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        if limit is not None:
            candidates = candidates[:limit]
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.4f} (std: {1:.4f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print()

# Number of iterations
n_iter_search = 200

In [ ]:
logreg = LogisticRegression(random_state=random_state)
rand_param = {
    'penalty': ['l1', 'l2'],
    'C': uniform(0.01, 10)
 }

logreg_search = RandomizedSearchCV(logreg, param_distributions=rand_param, n_iter=n_iter_search, cv=k_folds, n_jobs=4, verbose=1)
logreg_search.fit(X_train, y_train)
report(logreg_search.cv_results_)

logreg_best = logreg_search.best_estimator_

## 5.2.2 SVC

In [ ]:
svc = SVC(random_state=random_state, probability=True)
rand_param = {
    'C': uniform(0.01, 10),
    'gamma': uniform(0.01, 10)
 }

svc_search = RandomizedSearchCV(svc, param_distributions=rand_param, n_iter=n_iter_search, cv=k_folds, n_jobs=4, verbose=1)
svc_search.fit(X_train, y_train)
report(svc_search.cv_results_)

svc_best = svc_search.best_estimator_